In [52]:
# exam_scheduler_csp.py

import random
import time
from collections import defaultdict

# Variables
exams = ['Examen1', 'Examen2', 'Examen3', 'Examen4', 'Examen5', 'Examen6', 'Examen7']
days = ['lunes', 'martes', 'miércoles']
students = ['Estudiante1', 'Estudiante2', 'Estudiante3', 'Estudiante4']

# Cursos por estudiante
student_courses = {
    'Estudiante1': ['Examen1', 'Examen2', 'Examen3'],
    'Estudiante2': ['Examen2', 'Examen4', 'Examen5'],
    'Estudiante3': ['Examen3', 'Examen6', 'Examen7'],
    'Estudiante4': ['Examen4', 'Examen5', 'Examen6']
}

# Inicializar dominios
initial_domains = {exam: set(days) for exam in exams}

# AC-3 para consistencia de arcos
def ac3(domains, constraints):
    queue = [(xi, xj) for xi in exams for xj in exams if xi != xj]
    while queue:
        xi, xj = queue.pop(0)
        if revise(domains, xi, xj, constraints):
            if not domains[xi]:
                return False
            for xk in exams:
                if xk != xi and xk != xj:
                    queue.append((xk, xi))
    return True

def revise(domains, xi, xj, constraints):
    revised = False
    to_remove = set()
    for vi in domains[xi]:
        conflict = True
        for vj in domains[xj]:
            if vi != vj:
                conflict = False
                break
        if conflict:
            to_remove.add(vi)
    if to_remove:
        domains[xi] -= to_remove
        revised = True
    return revised

# Restricciones generales

def is_valid(assignment, constraints):
    for student, courses in constraints.items():
        assigned = [assignment[c] for c in courses if c in assignment]
        if len(assigned) != len(set(assigned)):
            return False
    return True

# Verificador de solución final completa

def is_complete_solution_valid(solution, constraints):
    if not solution:
        return False
    return is_valid(solution, constraints)

# Selección MRV

def select_exam(assignment, domains):
    unassigned = [var for var in exams if var not in assignment]
    return min(unassigned, key=lambda var: len(domains[var]))

# Backtracking

def backtracking(assignment, domains, constraints):
    if len(assignment) == len(exams):
        return assignment
    var = select_exam(assignment, domains)
    for value in sorted(domains[var]):
        assignment[var] = value
        if is_valid(assignment, constraints):
            result = backtracking(assignment, domains, constraints)
            if result:
                return result
        del assignment[var]
    return None

# Local Search helpers

def generate_initial_solution():
    return {exam: random.choice(days) for exam in exams}

def heuristic(assignment):
    cost = 0
    for student, courses in student_courses.items():
        assigned_days = [assignment[c] for c in courses]
        cost += len(assigned_days) - len(set(assigned_days))
    return cost

def generate_neighbors(solution):
    neighbors = []
    for exam in exams:
        for day in days:
            if day != solution[exam]:
                neighbor = solution.copy()
                neighbor[exam] = day
                neighbors.append(neighbor)
    return neighbors

def local_search():
    current = generate_initial_solution()
    while True:
        neighbors = generate_neighbors(current)
        best = min(neighbors, key=heuristic)
        if heuristic(best) >= heuristic(current):
            break
        current = best
    return current

# Beam Search

def beam_search(k=3):
    beam = [generate_initial_solution() for _ in range(k)]
    while True:
        candidates = []
        for sol in beam:
            candidates.extend(generate_neighbors(sol))
        candidates = sorted(candidates, key=heuristic)
        if heuristic(candidates[0]) == 0:
            return candidates[0]
        new_beam = candidates[:k]
        if all(heuristic(s) >= heuristic(beam[0]) for s in new_beam):
            break
        beam = new_beam
    return beam[0]

# Medición de tiempos y ejecución
if __name__ == "__main__":
    import copy
    
    print("\n--- Backtracking Search ---")
    start = time.time()
    ac3_domains = copy.deepcopy(initial_domains)
    ac3(ac3_domains, student_courses)
    back_sol = backtracking({}, ac3_domains, student_courses)
    print(f"Solution: {back_sol}")
    print(f"Valid: {is_complete_solution_valid(back_sol, student_courses)}")
    print(f"Time: {time.time() - start:.4f}s")

    print("\n--- Local Search ---")
    start = time.time()
    local_sol = local_search()
    print(f"Solution: {local_sol}, Conflicts: {heuristic(local_sol)})")
    print(f"Valid: {is_complete_solution_valid(local_sol, student_courses)}")
    print(f"Time: {time.time() - start:.4f}s")

    print("\n--- Beam Search ---")
    start = time.time()
    beam_sol = beam_search()
    print(f"Solution: {beam_sol}, Conflicts: {heuristic(beam_sol)})")
    print(f"Valid: {is_complete_solution_valid(beam_sol, student_courses)}")
    print(f"Time: {time.time() - start:.4f}s")


--- Backtracking Search ---
Solution: {'Examen1': 'lunes', 'Examen2': 'martes', 'Examen3': 'miércoles', 'Examen4': 'lunes', 'Examen5': 'miércoles', 'Examen6': 'martes', 'Examen7': 'lunes'}
Valid: True
Time: 0.0000s

--- Local Search ---
Solution: {'Examen1': 'miércoles', 'Examen2': 'lunes', 'Examen3': 'martes', 'Examen4': 'miércoles', 'Examen5': 'martes', 'Examen6': 'lunes', 'Examen7': 'miércoles'}, Conflicts: 0)
Valid: True
Time: 0.0010s

--- Beam Search ---
Solution: {'Examen1': 'lunes', 'Examen2': 'miércoles', 'Examen3': 'martes', 'Examen4': 'lunes', 'Examen5': 'martes', 'Examen6': 'miércoles', 'Examen7': 'lunes'}, Conflicts: 0)
Valid: True
Time: 0.0000s


### **Conclusiones:**

1. **Eficiencia en el tiempo de ejecución:**
   - **Backtracking Search** fue confiablemente exacto, pero su rendimiento puede degradarse en problemas más grandes. En este caso, resolvió el problema correctamente con un tiempo despreciable.
   - **Local Search** fue el más rápido (0.0010s), mostrando su ventaja en problemas pequeños donde se buscan soluciones aproximadas rápidamente.
   - **Beam Search** también fue veloz (0.0000s) y encontró soluciones válidas. Su rendimiento depende del parámetro `beam width`, que en este caso fue suficiente para resolver correctamente.

2. **Calidad de las soluciones:**
   - Todas las soluciones fueron **válidas y libres de conflictos** según las restricciones establecidas (ningún estudiante con dos exámenes el mismo día).
   - **Backtracking** garantiza una solución válida por su naturaleza exhaustiva.
   - **Local Search** y **Beam Search** también alcanzaron soluciones válidas en este problema, demostrando que pueden ser eficaces en CSPs simples o medianamente complejos.

3. **Comparación general:**
   - **Backtracking** asegura precisión pero no escalabilidad.
   - **Beam Search** ofrece un buen balance entre velocidad y calidad si se elige correctamente `beam_width`.
   - **Local Search** es el más eficiente en tiempo, ideal para problemas con soluciones aceptables pero no necesariamente óptimas.
 encontró soluciones válidas rápidamente, pero el valor de `beam_width` influye en la calidad de la solución. Con un **beam width** adecuado, el algoritmo puede encontrar una solución suficientemente buena sin un alto costo computacional, pero no siempre encontrará la mejor solución si el espacio de búsqueda es grande.
   - **Local Search** fue eficiente en términos de tiempo, pero es probable que no haya encontrado la mejor solución debido a su naturaleza de búsqueda local. A pesar de esto, en problemas donde el tiempo es crucial y la calidad de la solución no es lo más importante, este algoritmo podría ser la opción más efectiva.

3. **Comparación general:**
   - **Backtracking** es adecuado para problemas pequeños con pocas restricciones, ya que garantiza encontrar la solución óptima. Sin embargo, su rendimiento se ve afectado cuando el tamaño del problema aumenta significativamente.
   - **Beam Search** es adecuado para problemas más grandes, ya que reduce el espacio de búsqueda mediante el **beam width**, lo que mejora el rendimiento en términos de tiempo. La calidad de la solución puede verse comprometida si el **beam width** es demasiado pequeño, pero se puede ajustar para encontrar un balance entre tiempo y calidad.
   - **Local Search** es el algoritmo más rápido, pero tiene la desventaja de ser sensible a los óptimos locales. En problemas donde el tiempo es una prioridad y no se necesita la mejor solución, este algoritmo es la opción más eficiente.
